In [1]:
!pwd

/home/jupyter/drone/transfer


In [3]:
%cd .. 
%cd coco
!bash get_coco_dataset_images.sh

/home/jupyter/drone
/home/jupyter/drone/coco
--2019-08-23 17:43:19--  https://pjreddie.com/media/files/train2014.zip
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510435630 (13G) [application/zip]
Saving to: ‘train2014.zip’

train2014.zip       100%[===================>]  12.58G  18.3MB/s    in 8m 48s  

2019-08-23 17:52:07 (24.4 MB/s) - ‘train2014.zip’ saved [13510435630/13510435630]

--2019-08-23 17:52:07--  https://pjreddie.com/media/files/val2014.zip
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6645013297 (6.2G) [application/zip]
Saving to: ‘val2014.zip’

val2014.zip         100%[===================>]   6.19G  29.1MB/s    in 4m 13s  

2019-08-23 17:56:20 (25.1 MB/s) - ‘val2014.zip’ saved [6645013297/664501

In [4]:
!bash unzip_annotations.sh 

Fix archive (-F) - assume mostly intact archive
 copying: instances_train2014.json
  error:  invalid compressed data to inflate instances_train2014.json


In [1]:
import time
import glob
import torch
import os 
from IPython.display import Image, clear_output
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.1.0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [2]:
!pip install cython
!pip install nvidia-ml-py3
!pip install -U -r requirements.txt

  Stored in directory: /home/jupyter/.cache/pip/wheels/e4/1d/06/640c93f5270d67d0247f30be91f232700d19023f9e66d735c7
Successfully built nvidia-ml-py3
    100% |████████████████████████████████| 20.3MB 2.2MB/s 
Requirement already up-to-date: opencv-python in /opt/anaconda3/lib/python3.7/site-packages (from -r requirements.txt (line 7)) (4.1.0.25)
    100% |████████████████████████████████| 748.9MB 58kB/s 
Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(104, 'ECONNRESET')"))': /simple/matplotlib/
    100% |████████████████████████████████| 13.1MB 5.1MB/s 
    100% |████████████████████████████████| 1.5MB 24.5MB/s 
    100% |████████████████████████████████| 51kB 20.5MB/s 
    100% |████████████████████████████████| 3.8MB 11.6MB/s 
Requirement already up-to-date: future in /opt/anaconda3/lib/python3.7/site-packages (from -r requirements.txt (line 13)) (0.17.1)
    100% |███████████████

In [3]:
!sudo python3 train.py --cfg cfg/plane-drone.cfg --epochs 1 --batch-size 5 --data data/plane_drone_val.data

Namespace(accumulate=2, batch_size=5, bucket='', cache_images=False, cfg='cfg/plane-drone.cfg', data='data/plane_drone_val.data', epochs=1, evolve=False, img_size=416, img_weights=False, multi_scale=False, nosave=False, notest=False, num_workers=16, rect=False, resume=False, transfer=False, xywh=False)
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  154M  100  154M    0     0  29.9M      0  0:00:05  0:00:05 --:--:-- 36.2M
data/plane_drone_train.txt
[]
Model Summary: 222 layers, 6.15291e+07 parameters, 6.15291e+07 gradients

     Epoch   gpu_mem   GIoU/xy        wh       obj       cls     total   targets  img_size
       0/0      1.3G       0.2         0     0.713         5      5.91         1       416: 100%|█| 143/143 [00:35<00:00,  1.44it/s]
data/plane_drone_val.txt
[]
          

In [7]:
!sudo python3 test.py --data data/plane_drone_val.data --cfg cfg/plane-drone.cfg --weights weights/best.pt --img-size 416 --batch-size 5

Namespace(batch_size=5, cfg='cfg/plane-drone.cfg', conf_thres=0.001, data='data/plane_drone_val.data', img_size=416, iou_thres=0.5, nms_thres=0.5, save_json=False, weights='weights/best.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

data/plane_drone_val.txt
[]
                         Class    Images   Targets         P         R       mAP        F1: 100%|█| 61/61 [00:59<00:00,  1.07s/it]
                           all       301       418  0.000608    0.0352  4.98e-05   0.00119
                         drone       301       152   4.3e-05   0.00658  4.43e-07  8.54e-05
                      airplane       301       266   0.00117    0.0639  9.92e-05    0.0023


In [9]:
!sudo python3 -c "from utils import utils; utils.plot_results()"
#Image(filename='results.png', width=800)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/jupyter/drone/transfer/utils/utils.py", line 766, in plot_results
    n = results.shape[1]  # number of rows
IndexError: tuple index out of range
Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x7f7ecfda7c80>
Traceback (most recent call last):
  File "/usr/lib/python3.5/weakref.py", line 117, in remove
TypeError: 'NoneType' object is not callable


In [14]:
!python3 detect.py --weights weights/best.pt --images drone-vs-plane/images/val-test-all
#Image(filename='output/zidane.jpg', width=600)
%cd transfer

Namespace(cfg='cfg/yolov3-spp.cfg', conf_thres=0.3, data='data/coco.data', fourcc='mp4v', half=False, images='drone-vs-plane/images/val-test-all', img_size=416, nms_thres=0.5, output='output', webcam=False, weights='weights/best.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Traceback (most recent call last):
  File "detect.py", line 154, in <module>
    output=opt.output)
  File "detect.py", line 37, in detect
    model.load_state_dict(torch.load(weights, map_location=device)['model'])
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 845, in load_state_dict
    self.__class__.__name__, "\n\t".join(error_msgs)))
RuntimeError: Error(s) in loading state_dict for Darknet:
	Missing key(s) in state_dict: "module_list.85.Conv2d.weight", "module_list.85.BatchNorm2d.weight", "module_list.85.BatchNorm2d.bias", "module_list.85.BatchNorm2d.running_mean", "module_list.85.BatchNorm2d.running_var", "module_list.86